In [ ]:
!pip install tensorflow-text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from keras.models import load_model
from keras.optimizers import Adam

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import seaborn as sb
import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

In [ ]:
def load_dataset(dataset_name, filepath):
  df = pd.read_csv(filepath)
  return df, dataset_name

def split_data(df):
    test_size = 0.20
    x = np.array(df["text"])
    y = np.array(df["class"])

    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = test_size, random_state=42) #random state ensure same sample
    print("Train Set :", x_train.shape, y_train.shape)
    print("Test Set  :", x_test.shape, y_test.shape)
    print("Total ", len(df))
    # y in digit form
    y_train_binary = np.array(list(map(lambda x:1 if x=="Hate" else 0, y_train)))
    y_test_binary = np.array(list(map(lambda x:1 if x=="Hate" else 0, y_test)))
    return x_train, y_train, y_train_binary, x_test, y_test, y_test_binary

def get_classification_report(i, cr):
    return [i, cr['accuracy'], cr['macro avg']['precision'],
            cr['macro avg']['recall'], cr['macro avg']['f1-score'],
            cr['Hate']['f1-score'],cr['Non-Hate']['f1-score'],
            cr['Hate']['support'],cr['Non-Hate']['support']]

def get_result_table():
    c = ['Model', 'Accuracy', 'precision', 'recall', 'f1-score', 'hate f1', "non-hate f1", 'hate support', 'non-hate support']
    result_table = pd.DataFrame(columns=c)
    return result_table

def get_result_single(y_test, y_test_pred, model_name, result_table):
    cr = classification_report(y_test, y_test_pred, labels=["Hate","Non-Hate"], output_dict=True)
    result_table.loc[len(result_table)] = get_classification_report(model_name, cr)

METRICS = [
    tf.keras.metrics.BinaryAccuracy(name="accuracy"),
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall")
]

def nn_predict(model,x_test, y_test_binary):
    score = model.evaluate(x_test, y_test_binary, verbose=0)
    print("Score: ", score[0])
    print("Accuracy: ", score[1])

    y_test_pred_percent = model.predict(x_test, verbose=0)
    y_test_pred = np.where(y_test_pred_percent > 0.5, "Hate", "Non-Hate")
    y_test_pred = y_test_pred.flatten()

    return y_test_pred

In [ ]:
def bert(variant, x_train, y_train_binary, x_test, y_test_binary):
  # bert layers that process the text
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
  preprocessed_text = bert_preprocess(text_input)
  outputs = bert_encoder(preprocessed_text)
  bert_type = ""
  if(variant == 1):
    # base bert
    bert_type = "base"
    l = tf.keras.layers.Dropout(0.2)(outputs["pooled_output"])
  else:
    if(variant == 2):
      # bert_cnn
      bert_type = "cnn"
      l = tf.keras.layers.Conv1D(128,3,activation='relu')(outputs["sequence_output"])
      l = tf.keras.layers.GlobalMaxPooling1D()(l)
    elif(variant == 3):
      # bert_rnn
      bert_type = "rnn"
      l = tf.keras.layers.SimpleRNN(128)(outputs["sequence_output"])
    elif(variant == 4):
      # bert_lstm
      bert_type = "lstm"
      l = tf.keras.layers.LSTM(128)(outputs["sequence_output"])
    elif(variant == 5):
      # bert_gru
      bert_type = "gru"
      l = tf.keras.layers.GRU(128)(outputs["sequence_output"])

    l = tf.keras.layers.Dropout(0.2)(l)

  l = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(l)

  #construct final model
  model = tf.keras.Model(inputs=[text_input], outputs=[l])

  opt = Adam()
  model.compile(optimizer=opt,
                loss='binary_crossentropy',
                metrics=METRICS)

  h = model.fit(x_train, y_train_binary, epochs=3,
          validation_data = (x_test, y_test_binary),
          batch_size = 32,
          )
  return model, h, bert_type

def base_bert_train_dataset_all(df, dataset_name, df_result):
  x_train, y_train, y_train_binary, x_test, y_test, y_test_binary = split_data(df)
  df['hate'].value_counts()

  model, h, bert_type = bert(1, x_train, y_train_binary, x_test, y_test_binary)
  y_test_pred = nn_predict(model, x_test, y_test_binary)
  get_result_single(y_test, y_test_pred, dataset_name+"_bert", df_result)

def bert_special_train_all(df, dataset_name, df_result):
  x_train, y_train, y_train_binary, x_test, y_test, y_test_binary = split_data(df)
  df['hate'].value_counts()

  # cnn
  model, h, bert_type = bert(2, x_train, y_train_binary, x_test, y_test_binary)
  y_test_pred = nn_predict(model, x_test, y_test_binary)
  get_result_single(y_test, y_test_pred, dataset_name+"_bert_"+bert_type, df_result)

  # rnn
  model, h, bert_type = bert(3, x_train, y_train_binary, x_test, y_test_binary)
  y_test_pred = nn_predict(model, x_test, y_test_binary)
  get_result_single(y_test, y_test_pred, dataset_name+"_bert_"+bert_type, df_result)

  # lstm
  model, h, bert_type = bert(4, x_train, y_train_binary, x_test, y_test_binary)
  y_test_pred = nn_predict(model, x_test, y_test_binary)
  get_result_single(y_test, y_test_pred, dataset_name+"_bert_"+bert_type, df_result)

  # gru
  model, h, bert_type = bert(5, x_train, y_train_binary, x_test, y_test_binary)
  y_test_pred = nn_predict(model, x_test, y_test_binary)
  get_result_single(y_test, y_test_pred, dataset_name+"_bert_"+bert_type, df_result)


In [ ]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

bert_preprocess = hub.KerasLayer(preprocess_url, name="preprocessing")
bert_encoder = hub.KerasLayer(encoder_url, name="bert_encoder")

In [ ]:
df_result = get_result_table()

In [ ]:
# gab dataset
df, dataset_name = load_dataset("gab", "data_final.csv")
base_bert_train_dataset_all(df, dataset_name, df_result)

Train Set : (21715,) (21715,)
Test Set  : (5429,) (5429,)
Total  27144
Epoch 1/3
679/679 [==============================] - 334s 474ms/step - loss: 0.3666 - accuracy: 0.8741 - precision: 0.4394 - recall: 0.0106 - val_loss: 0.3164 - val_accuracy: 0.8864 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/3
679/679 [==============================] - 295s 435ms/step - loss: 0.3427 - accuracy: 0.8750 - precision: 0.5484 - recall: 0.0249 - val_loss: 0.3024 - val_accuracy: 0.8895 - val_precision: 0.8696 - val_recall: 0.0324
Epoch 3/3
679/679 [==============================] - 322s 475ms/step - loss: 0.3337 - accuracy: 0.8751 - precision: 0.5362 - recall: 0.0407 - val_loss: 0.3023 - val_accuracy: 0.8943 - val_precision: 0.7590 - val_recall: 0.1021
Score:  0.3022642731666565
Accuracy:  0.8942714929580688


In [ ]:
# implicit dataset
df, dataset_name = load_dataset("implicit", "data_final.csv")
base_bert_train_dataset_all(df, dataset_name, df_result)

Train Set : (17182,) (17182,)
Test Set  : (4296,) (4296,)
Total  21478
Epoch 1/3
537/537 [==============================] - 273s 503ms/step - loss: 0.6635 - accuracy: 0.6165 - precision: 0.4961 - recall: 0.2508 - val_loss: 0.6363 - val_accuracy: 0.6455 - val_precision: 0.5435 - val_recall: 0.4173
Epoch 2/3
537/537 [==============================] - 233s 434ms/step - loss: 0.6464 - accuracy: 0.6327 - precision: 0.5355 - recall: 0.2829 - val_loss: 0.6197 - val_accuracy: 0.6562 - val_precision: 0.5958 - val_recall: 0.2953
Epoch 3/3
537/537 [==============================] - 269s 501ms/step - loss: 0.6342 - accuracy: 0.6407 - precision: 0.5536 - recall: 0.3017 - val_loss: 0.6242 - val_accuracy: 0.6418 - val_precision: 0.6853 - val_recall: 0.1054
Score:  0.6242021322250366
Accuracy:  0.6417597532272339


In [ ]:
# se2019 dataset
df, dataset_name = load_dataset("se", "data_final.csv")
base_bert_train_dataset_all(df, dataset_name, df_result)

Train Set : (10384,) (10384,)
Test Set  : (2596,) (2596,)
Total  12980
Epoch 1/3
 87/325 [=======>......................] - ETA: 1:05:47 - loss: 0.7076 - accuracy: 0.5478 - precision: 0.4415 - recall: 0.2662

In [ ]:
# balanced dataset
df, dataset_name = load_dataset("balanced", "data_final.csv")
base_bert_train_dataset_all(df, dataset_name, df_result)

Train Set : (27178,) (27178,)
Test Set  : (6795,) (6795,)
Total  33973
Epoch 1/3
850/850 [==============================] - 385s 448ms/step - loss: 0.6836 - accuracy: 0.5890 - precision: 0.5897 - recall: 0.5875 - val_loss: 0.6398 - val_accuracy: 0.6509 - val_precision: 0.6407 - val_recall: 0.6809
Epoch 2/3
850/850 [==============================] - 380s 447ms/step - loss: 0.6566 - accuracy: 0.6117 - precision: 0.6120 - recall: 0.6147 - val_loss: 0.6417 - val_accuracy: 0.6414 - val_precision: 0.5969 - val_recall: 0.8623
Epoch 3/3
850/850 [==============================] - 378s 445ms/step - loss: 0.6489 - accuracy: 0.6230 - precision: 0.6243 - recall: 0.6219 - val_loss: 0.6418 - val_accuracy: 0.6387 - val_precision: 0.5928 - val_recall: 0.8771
Score:  0.6418291926383972
Accuracy:  0.6387049555778503


In [ ]:
df_result

,Model,Accuracy,precision,recall,f1-score,hate f1,non-hate f1,hate support,non-hate support
0,gab_bert,0.894272,0.827704,0.548975,0.561746,0.180000,0.943493,617,4812
1,implicit_bert,0.641760,0.662160,0.537869,0.476647,0.182687,0.770607,1632,2664
2,se_bert,0.618259,0.638948,0.571686,0.536737,0.342402,0.731072,1121,1475
3,balanced_bert_,0.638705,0.680054,0.639648,0.617609,0.707425,0.527794,3384,3411


In [ ]:
df_result = get_result_table()

In [ ]:
# balanced dataset on special bert
# cnn, rnn, lstm, gru
df, dataset_name = load_dataset("balanced", "data_final.csv")
bert_special_train_all(df, dataset_name, df_result)

Train Set : (27178,) (27178,)
Test Set  : (6795,) (6795,)
Total  33973
Epoch 1/3
850/850 [==============================] - 399s 445ms/step - loss: 0.6833 - accuracy: 0.5736 - precision: 0.5746 - recall: 0.5735 - val_loss: 0.6422 - val_accuracy: 0.6431 - val_precision: 0.6147 - val_recall: 0.7595
Epoch 2/3
850/850 [==============================] - 365s 430ms/step - loss: 0.6580 - accuracy: 0.6121 - precision: 0.6133 - recall: 0.6109 - val_loss: 0.6281 - val_accuracy: 0.6624 - val_precision: 0.6653 - val_recall: 0.6483
Epoch 3/3
850/850 [==============================] - 365s 430ms/step - loss: 0.6495 - accuracy: 0.6244 - precision: 0.6254 - recall: 0.6244 - val_loss: 0.6376 - val_accuracy: 0.6262 - val_precision: 0.7041 - val_recall: 0.4303
Score:  0.6375505924224854
Accuracy:  0.6261957287788391
Train Set : (27178,) (27178,)
Test Set  : (6795,) (6795,)
Total  33973
Epoch 1/3
850/850 [==============================] - 386s 446ms/step - loss: 0.5839 - accuracy: 0.6825 - precision: 0.69

In [ ]:
df_result

,Model,Accuracy,precision,recall,f1-score,hate f1,non-hate f1,hate support,non-hate support
0,balanced_bert_,0.626196,0.648096,0.625420,0.611000,0.534116,0.687884,3384,3411
1,balanced_bert_cnn,0.747314,0.747530,0.747369,0.747284,0.750036,0.744532,3384,3411
2,balanced_bert_rnn,0.677704,0.700006,0.677034,0.667943,0.611012,0.724874,3384,3411
3,balanced_bert_lstm,0.731567,0.731624,0.731595,0.731563,0.732630,0.730496,3384,3411
4,balanced_bert_gru,0.742605,0.742843,0.742663,0.742569,0.745600,0.739538,3384,3411
